In [17]:
import os
import re
from get_input import get_input

os.chdir('/home/soup/Desktop/Projects/advent-of-code')

In [15]:
input_str = get_input(2015, 12)

Found a datafile!


In [18]:
pattern = r'[{}\[\]]|\bred\b'

matches = re.findall(pattern, input_str)
output_str = ''.join(matches)

output_str

'{{{{}{{[red{}[]]}{}[[]{red[redred][red]}red]}{{red{}{[red]}}[]}{[{}]{red[][{}[]]}}}[[{{}}[[[red]{}{red}{red}]]][[[]{[]{}{}{}}red][{[redredred][]}[[]]red{[red]}]]]}{[{[]{{}}}[[[{}{}{}]{{}redred}]{[[]]{}[[red]][[][red]]{[]{}[]}red{}}{{}{}{}}][[red][[[]]]{}[red]]{[]}][[[]{redred[[][]]}{}[{red}[{redred}[redred]]]{}][[red]]{[[red{red}]{{}}[][[]][redred[]]]}][{[{}][[[redred]]]{redred}[[[red]]]{}red{[]}}]{[{red{red{}}{{}[]red}}[]]{[{}][redred{red}[{red}][[]]]{}{[][red]{}{red}}}{{{red}[][][]}[{redredred{red}}[[redred][]{}]]{{[][red]}{}{{}[redred]}{}[red]}red{}}[[red{[]}]][[]][[[red{red}]{redred}][{red[]}{}]{}[[{red}red]]][{red}]}[[{}{[[red][redred][red]]}]{{[[red]]}}[[[]{}][]{{red}[][red[][redred]]}]]}{[{{[[red]red]}{{{red}}}[[[]{}]]}[{}{[{red}]}red[[[]red]redred[[][]{red}]][{red}][red[[]][{}]]][{red{[][red]}{{}red{}}}][red{[[][]]{[]red{red}}{}}[{}{red{}[redredredred][red]}][[][{}]]{}{{[]}}]{{{}{redred}{red[]red{}{}}[]}{}}]}[[][{[{redred}{[redred]{red}}]}[][{{}[]red}{red{[][red]}}[[red{red}][